# DEEPX 튜토리얼 04 - DX‑STREAM 워크플로우

이 튜토리얼은 DX‑STREAM 개요와, 사용자가 만든 AI 모델을 DX‑STREAM 으로 실행하기 위한 end-to-end workflow 를 소개합니다.

**학습 목표**:
- DX‑STREAM 이 무엇인지 이해하고, DEEPX SDK 에서 어떻게 지원되는지 이해하기
- 핵심 Plugin 학습 (DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd)
- Custom AI 모델을 지원하기 위한 전/후처리 라이브러리 구현하기

>이 튜토리얼은 2025년 9월 공개된 dx‑all‑suite v2.0.0 을 기반으로 합니다.


## 0. DX‑STREAM Overview
DEEPX NPU 에서 Vision AI 를 위한 GStreamer Plugin 들의 모음.
```text
Source → (decodebin) → DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd → Sink
```
- **DxPreprocess**: 리사이즈/정규화/레이아웃 변환 등 모델 입력에 맞는 전처리
- **DxInfer**: DX‑RT를 통해 NPU 추론 수행
- **DxPostprocess**: 모델 출력 해석 (예: YOLO 디텍션, 세그멘테이션 등)
- **DxTracker**: (선택) 다중 객체 추적
- **DxOsd**: 박스/라벨/마스크 등 오버레이

  ![img](assets/dx-stream-pipeline.png)

> 더 자세한 사항은 다음 DX-Stream User Guide 참고 👉 [here](https://developer.deepx.ai/?files=MjUxNw==)!


## 1. 사전 준비 사항:
DX-STREAM 이 잘 설치됐는지 재확인하는 부분으로, 설치는 Tutorial 01 (T01) 에서 완료됨을 가정하고 진행됩니다.

아래 스크립트로 `dx‑stream` 경로로 이동 (루트 경로는 `ROOT_PATH` 환경변수값에 저장).

In [ ]:
# "dx-tutorials/dx-all-suite/dx-runtime/dx_stream" 디렉터리로 이동
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-runtime/dx_stream

다음 명령으로 사용될 **모델과 샘플 비디오**를 다운로드합니다:

In [ ]:
# Assets (모델/비디오)을 다운로드 합니다.
!./setup.sh

다음 명령으로 잘 다운로드 됐는지 확인합니다. 

In [ ]:
# Demo 에 필요한 model/video file 들이 저장되는 위치에서 tree 명령어로 확인
!tree ${ROOT_PATH}/dx-all-suite/workspace/res

GStreamer에 DX‑STREAM 플러그인이 정상 등록되었는지 확인합니다: (아래 plugin 들이 설치되어야 함)
  - dxgather: DxGather
  - dxinfer: DXInfer
  - dxinputselector: DXInputSelector
  - dxmsgbroker: DXMsgBroker
  - dxmsgconv: DXMsgConv
  - dxosd: DXOsd
  - dxoutputselector: DXOutputSelector
  - dxpostprocess: DXPostprocess
  - dxpreprocess: DXPreprocess
  - dxrate: DXRate
  - dxtracker: DXTracker

In [ ]:
# gst-inspect-1.0 으로 dxstream 관련 플러그인이 설치됐는지 확인
!gst-inspect-1.0 dxstream || echo 'dxstream plugin not found'

## 2. DX_STREAM 일단 실행해 보기 (run_demo.sh)

### 2.1. Video 인풋으로 YOLOv7 pipeline 실행해 보기
   - `Note`: 상단의 stop button ('■') 을 눌러서 실행중인 cell 을 강제 종료 가능!

In [ ]:
# 입력 비디오 파일 경로
VIDEO_SRC='dx_stream/samples/videos/doughnut.mp4'

# GStreamer 파이프라인 설정
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=dx_stream/configs/Object_Detection/YoloV7/inference_config.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

### 2.2. 위에서 사용한 각 DX plugin 들의 정의에 대해 **gst-inspect-1.0** 을 활용해 알아보기

In [ ]:
# Check dxpreprocess plugin (DXPreprocess – Performs input preprocessing for AI inference)
!gst-inspect-1.0 dxpreprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxinfer plugin (DXInfer – Executes AI model inference using the DEEPX NPU)
!gst-inspect-1.0 dxinfer
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/inference_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxpostprocess plugin (DXPostprocess – Processes and formats inference output)
!gst-inspect-1.0 dxpostprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxosd plugin (Overlays inference results onto video frames)
!gst-inspect-1.0 dxosd

## 3. Run Demo Script

dx_stream 경로에 **`run_demo.sh`** script 파일이 있으며 이 스크립트를 실행하여 DX Stream 의 다양한 예제 case 를 쉽게 테스트 가능하다.

이 스크립트 안에서 사용자의 0부터 7까지의 인풋을 받아 실제 데모를 수행하는 스크립트를 내부적으로 호출하게 된다.

In [ ]:
# Check the demo list - Total 8 GStreamer demos
!tail -n 15 run_demo.sh

사용자 인풋 '0' ~ '7' 의 값을 넣어서 run_demo.sh 를 실행해 보고 각 데모 케이스에 대해 알아보자.
   - `Note`: 상단의 stop button ('■') 을 눌러서 실행중인 cell 을 강제 종료 가능!

### 3.1. Single_network/object_detection - YOLOv7
![img](assets/pipline-single-detection.png)

In [ ]:
# 사용자 입력 '0' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 0 # single_network/object_detection

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/single_network/object_detection/run_YOLOV7.sh

### 3.2. Single_network/object_detection - YOLOv5 Face Detection

In [ ]:
# 사용자 입력 '1' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 1 # single_network/face_detection

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/single_network/face_detection/run_YOLOFACE.sh

사용자 인풋을 비디오가 아닌 카메라로 바꾸서 실행해 보자

In [ ]:
# 인풋을 USB webcam 의 원본 이미지 (RAW) 를 받는 경우
!gst-launch-1.0 \
  v4l2src device=/dev/video0 do-timestamp=true ! \
  videoconvert ! video/x-raw,width=1920,height=1080 ! queue ! \
  dxpreprocess config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/preprocess_config.json ! queue ! \
  dxinfer config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/inference_config.json  ! queue ! \
  dxpostprocess config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/postprocess_config.json ! queue ! \
  dxosd width=1920 height=1080 ! queue ! \
  videoconvert ! fpsdisplaysink sync=false

In [ ]:
# 인풋을 USB webcam 의 압축된 이미지 (MJPEG) 를 받는 경우 - USB webcam 이 기능 지원하는 경우 실행 가능
!gst-launch-1.0 \
  v4l2src device=/dev/video0 do-timestamp=true ! \
  image/jpeg,width=1920,height=1080,framerate=30/1 ! jpegdec ! videoconvert ! queue ! \
  dxpreprocess config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/preprocess_config.json ! queue ! \
  dxinfer config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/inference_config.json  ! queue ! \
  dxpostprocess config-file-path=dx_stream/configs/Face_Detection/YOLOV5S_Face/postprocess_config.json ! queue ! \
  dxosd width=1920 height=1080 ! queue ! \
  videoconvert ! fpsdisplaysink sync=false

### 3.3. Single_network/object_detection - Pose Estimation

In [ ]:
# 사용자 입력 '3' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 3 # single_network/pose_estimation

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/single_network/pose_estimation/run_YOLOPOSE.sh

### 3.4. Single_network/object_detection - Segmentation

In [ ]:
# 사용자 입력 '4' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 4 # single_network/semantic_segmentation

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/single_network/semantic_segmentation/run_DeepLabV3PlusMobileNetV2.sh

### 3.5. Tracker - YOLOV5S tracker
![img](assets/pipline-single-tracking.png)

In [ ]:
# 사용자 입력 '2' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 2 # tracking/run_YOLOV5S_tracker

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/tracking/run_YOLOV5S_tracker.sh

### 3.6. Multi-Stream - YOLOv5S
![img](assets/pipeline-multi-stream.png)

In [ ]:
# 사용자 입력 '5' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 5 # multi_stream/run_multi_stream_YOLOV5S

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/multi_stream/run_multi_stream_YOLOV5S.sh


<img src="assets/pipeline-multi-stream-single-infer.png" style="max-width: 1400px;">

입력을 multi channel 로 받지만 사용하는 AI 모델은 하나인 경우에는 위처럼 **dxinfer** plugin 을 각 입력마다 배치 시킬 필요가 없다. 추론하는 pipeline 을 하나로 해서 공유해서 사용하게 되면 NPU 내부의 DRAM 공간을 절약하면서 보다 효율적으로 모델 동작이 가능하다.

M1 의 경우 4GB 의 DRAM size 를 가지고 있는데, 사용하는 모델의 사이즈가 1.1GB 인 경우 위와같이 4개의 dxinfer 를 배치하게 되면 M1 의 DRAM 용량을 초과하게 되어 수행이 불가하게 된다. 따라서 이 경우 multi-stream & single inference 구조로 파이프라인 설계를 권장한다.

In [ ]:
!dx_stream/pipelines/multi_stream/run_multi_stream_single_infer_YOLOV5S.sh

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/multi_stream/run_multi_stream_single_infer_YOLOV5S.sh

### 3.7. Multi-Stream - RTSP

In [ ]:
# 사용자 입력 '6' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 6 # rtsp/run_RTSP

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/rtsp/run_RTSP.sh

### 3.8. Secondary Mode
![img](assets/pipeline-secondary.png)

- **Primary Mode** applies preprocessing to the entire frame. If object detection is performed within the
same pipeline, DxPreprocess will operate in this mode.
- **Secondary Mode** applies preprocessing to individual object regions detected in this frame. This mode
requires object metadata (e.g., from an upstream object detection element).

In [ ]:
# 사용자 입력 '7' 을 주어 데모를 실행하자.
!./run_demo.sh <<< 7 # secondary_mode/run_secondary_mode

In [ ]:
# GStreamer pipelie 이 어떻게 구성 됐는지 살펴보자.
!cat dx_stream/pipelines/secondary_mode/run_secondary_mode.sh

## 4. Custom 모델로 DX Stream 을 활용해 Applicaiton 만들기

![img](assets/custom-pipeline.png)

이번 장에서는 DX-STREAM 파이프라인 안에 내가 만든 AI 모델을 넣고, 그 모델에 맞는 자체 후처리 로직을 추가하는 방법을 설명합니다.

DxPreprocess, DxInfer, DxPostprocess처럼 DX-STREAM에서 제공하는 모듈들을 조합하여 커스텀 기능을 손쉽게 넣는 방법을 집중적으로 다룹니다.

또한, 이전 Tutorial-03 에서 사용했던 지게차와 작업자를 감지하는 모델을 예제로 다시 사용하며,이에 맞춰 dxpostprocess 플러그인을 직접 수정하는 작업을 진행합니다.

<img src="assets/detection-goal.jpg" style="max-width: 1400px;">

### 4.1. Tutorial 03 에서 사용된 compiled DXNN file 을 아래 명령어를 통해 다운로드

In [ ]:
!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.dxnn"

### 4.2. `dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp` 수정

기존의 YOLOv7 용 post-processing 을 custom model 에 맞게 수정하자.

YOLOv7 Custom model 에서 input 쪽 수정은 없고 class 만 변경된 사항이기 때문에 pre-processing 단 수정은 필요없고 post-processing 단만 수정이 필요하다. 

In [ ]:
DIFF_TEXT = r"""diff --git a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
index 7d048e9..26125ca 100644
--- a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
+++ b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
@@ -58,21 +58,11 @@ struct YoloConfig {
     float nms_threshold = 0.4f;      // IoU threshold for NMS

     // Number of classes in your dataset
-    int num_classes = 80;
+    int num_classes = 2;

     // COCO dataset class names (modify for your dataset)
     std::vector<std::string> class_names = {
-        "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
-        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
-        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
-        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
-        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
-        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
-        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
-        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
-        "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
-        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
-        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
+        "Forklift", "Worker"
     };
 };

"""

# 파일로 저장
with open("dx_stream_update.diff", "w", encoding="utf-8") as f:
    f.write(DIFF_TEXT)

print("Saved: dx_stream_update.diff (bytes)", len(DIFF_TEXT))

### 4.3. 수정한 내용을 적용하기
`git apply`로 수정사항을 반영합니다.

In [ ]:
#!git checkout -- dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
!git apply --whitespace=fix dx_stream_update.diff

### 4.4. DX_STREAM 을 다시 빌드하기

In [ ]:
!source ../venv-dx-runtime/bin/activate && ./build.sh

### 4.5. 커스텀 모델 (yolov7‑forklift‑person.dxnn) 을 불러올 **JSON** 작성

In [ ]:
import json, os
yolov7_custom = {
    "preprocess_id": 1,
    "inference_id": 1,
    "model_path" : "./yolov7-forklift-person.dxnn"
}
with open("yolov7-forklift-person.json", "w") as f: json.dump(yolov7_custom, f, indent=2)

### 4.6. 테스트용 비디오 다운로드

In [ ]:
!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/forklift-worker.mp4"

### 4.7. 아래 DX‑STREAM 파이프라인 실행

In [ ]:
# 입력 비디오 파일 경로
VIDEO_SRC='forklift-worker.mp4'

# GStreamer 파이프라인 설정
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=yolov7-forklift-person.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

## 5. 디버그 Tips

GStreamer 파이프라인 구성을 이미지로 덤프하기:

In [ ]:
!sudo apt install -y graphviz

In [ ]:
!rm *.dot
!export GST_DEBUG_DUMP_DOT_DIR=./ && ./run_demo.sh <<< 0

In [ ]:
!ls *.dot

In [ ]:
#!dot -Tpng -o NULL_READY.png *NULL_READY.dot
#!dot -Tpng -o READY_PAUSED.png *READY_PAUSED.dot
!dot -Tpng -o PAUSED_PLAYING.png *PAUSED_PLAYING.dot
#!dot -Tpng -o PLAYING_PAUSED.png *PLAYING_PAUSED.dot
# -Tpng : 출력 형식을 png로 지정
# -o : 출력 파일 이름
# *.dot : 입력 파일 이름


In [ ]:
from IPython.display import Image, display

# 생성된 이미지를 표시
display(Image(filename="PAUSED_PLAYING.png"))